In [ ]:
using Plots
using StatsBase
using LinearAlgebra
using Statistics
using JLD2
using Dates
using LsqFit
using StaticArrays
using JSON

# data structure module (needs to be module to avoid overwriting local variables in REPL scope)
include("SpinSimParams.jl")
using .SpinSimParams

include("MTools.jl");
using .MTools

# function libraries
include("spin_sims.jl");

In [52]:
# interaction
α = 0.2#[0,0.05,0.075,0.10,0.125,0.15];#collect(LinRange(0.1, 0.2, 5));
ω = 0#collect(LinRange(0.0, 0.5, 11));

# flip operators
Ix = @SMatrix [0 1/2; 1/2 0];
f = [0.05,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,1.95]
UL90 = ();
UR90 = ();
for idx = 1:length(f)
    UL90 = (UL90..., exp(-1im*pi*f[idx]*Ix/2));
    UR90 = (UR90..., exp(1im*pi*f[idx]*Ix/2));
end

# number of frequencies
nx = 20;
ny = 20;
nz = 1;
n = (nx, ny, nz);
nfreq = prod(n);

# make the parameter file
params = make_params(α, ω, n);

params["τ"] = 100e-6
params["post_90_n"] = 2;
params["bw"] = 0.5/1;
params["σ"] = 0.05/1;

params["UL90"] = UL90;
params["UR90"] = UR90;
params["pulse_str"] = f;
params["dt"] = 1;
println(α)
println(f)
#println(ω[1:2])

gam_0 = 1e-3
gam_1 = gam_0
gam_2 = gam_0
gam_3 = 0.0

Iz = @SMatrix [1/2 0; 0 -1/2];

nJ = 3
Lj_list = []
push!(Lj_list, sqrt(gam_1)* @SMatrix [0 1; 0 0]) # down to up flip
push!(Lj_list, sqrt(gam_2)* @SMatrix [0 0; 1 0]) # up to down flip
push!(Lj_list, sqrt(gam_3)*Iz) # phase decoherence

params["Lj"] = Lj_list;
params["local_M"] = false;

# set the variables to loop over
params["integrated_sampling"] = true;
vars = [("UL90","UR90","pulse_str")]#["α", ("UL90", "UR90", "pulse_str")];
params["vars"] = vars;

0.2
[0.05, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 1.95]


In [53]:
I, d = make_idx(vars, params)
M_list = Array{Any}(undef, d)
save_dat = false

for i in I
    # generate temporary parameters
    tparams = make_temp_params(params, vars, i)

    # simulate
    #M_list[i] = spin_echo_sim(tparams)
    M_list[i] = spin_echo_sim_liouville(tparams)
    if (i[1] == 1)
        #println(string(i[2]))#,"/",length(params[vars[2]])))
    end

end

params["M_list"] = M_list;
using JSON

fname = "spinecho_pulse_a_w_small_07-15-2020.json"
if save_dat
    open(fname,"w") do f
        JSON.print(f, params)
    end
end

In [55]:
using FFTW
using LaTeXStrings

#tar_alph = 5;
#println(α[tar_alph])
plt1 = plot()
plt2 = plot()
ν0 = params["ν0"]

n_keep = 1;
dt_start = 300;
dt_end = 300;

# loop over the pulse strengths
for f0 = 2:length(f)-1
    
    # load in the M data
    M0 = convert(Array{Complex{Float64}},M_list[f0])
    nt = size(M0, 1);

    # select a truncated window to FFT
    M0_trunc = M0[convert(Int64,round(nt/3 + dt_start)):convert(Int64,round(nt - dt_end))]
    t_max = n_keep*100; # 200 micro seconds
    t = LinRange(0, t_max, size(M0_trunc, 1));
    dt = t[2]-t[1];

    # figure out the "frequency axis" after the FFT
    Fs = 1/dt; # 
    N = length(M0_trunc)
    freq_ax = (((0:N-1) .+ 1/2)/N .- 1/2)*Fs;

    # labeling
    ang_here = (f[f0]*90);
    P0 = f[1]*0.5;
    pulse_str = 10*log(f[f0]/P0)
    db_str = latexstring("\\textrm{",round(pulse_str,digits=1)," dB}")
    ang_str = latexstring(round(ang_here),"^\\circ");
    color_here = RGB(f0/length(f),1-f0/length(f),0.5)
    
    # plot the time-domian results
    M0_trunc_abs = broadcast(abs,M0_trunc) # normalize
    M0_trunc_abs = M0_trunc_abs/maximum(M0_trunc_abs)
    plot!(plt1,t, M0_trunc_abs .+ 1.5*(length(f)-f0),label="",color=color_here,linewidth=2, xlim=[t[1],t[length(t)]])
    plot!(plt1,t, 0*M0_trunc_abs .+ 1.5*(length(f)-f0),color=RGB(0,0,0),linestyle=:dash,label="")
    
    # FFT!
    M0_fft = fftshift(fft(M0_trunc))

    M0_fft = M0_fft/maximum(broadcast(abs,M0_fft)) # normalize
    plot!(plt2,freq_ax .+ ν0, 0*broadcast(abs,M0_fft) .+ 1.5*(length(f)-f0),color=RGB(0,0,0),linestyle=:dash,label="")
    plot!(plt2,freq_ax .+ ν0,broadcast(abs,M0_fft) .+ 1.5*(length(f)-f0),xlim=ν0 .+ [-0.4,0.4],label="",color=color_here,linewidth=2)

    annotate!(plt2,10.42,0.5+1.5*(length(f)-f0),text(db_str,color_here, :right,12))
    annotate!(plt2,9.65,0.5+1.5*(length(f)-f0),text(ang_str,color_here, :left,12))

end

plot!(plt2,ν0 .+ [0,0],[0, 1.5*(length(f)+1)],color=RGB(0,0,0),label="",linestyle=:dot)

xlabel!(plt1,L"\textrm{time}")
ylabel!(plt1,L"|M|")
xlabel!(plt2,L"\textrm{frequency (MHz)}")
ylabel!(plt2,L"|\tilde{M}|")

plot!(plt1,ylim=[1,15])
plot!(plt2,ylim=[1,15])

title!(plt1,L"\textrm{Signal (rotating frame)}")
title!(plt2,L"\textrm{Spectrum}")

plot(plt1,plt2)
savefig("power_sweep_0p200_alpha_1e-3_gam0_09-02-2020.svg")
